# Data Cleaning
-> remove `negative values` (`null`, `NaN`, `0`, .. ) by the `positive` (`mean`, `max`, `min`, ...)  
**very important step in data analysis and machine learning**!


In [1]:
import pandas as pd

data = pd.read_csv('dirty_data.csv')

data

,A,B,C
0,1000,0,3.0
1,400,5,6.0
2,700,6,NaN
3,100,11,12.0
4,1300,0,15.0
5,1600,17,NaN


## **Step1: Checking the data**

In [2]:
# .info() 
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       6 non-null      int64  
 1   B       6 non-null      int64  
 2   C       4 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 276.0 bytes


In [3]:
# .describe()
data.describe()

,A,B,C
count,6.000000,6.000000,4.000000
mean,850.000000,6.500000,9.000000
std,561.248608,6.595453,5.477226
min,100.000000,0.000000,3.000000
25%,475.000000,1.250000,5.250000
50%,850.000000,5.500000,9.000000
75%,1225.000000,9.750000,12.750000
max,1600.000000,17.000000,15.000000


In [4]:
# .isnull().sum() / isna.sum() -> check null 
print("check: nulls")
print("============")
data.isna().sum()

check: nulls


A    0
B    0
C    2
dtype: int64

## **Step2: Handle missing values**

In [5]:
# .dropna(inplace=True) -> remove the rows that contain null value

# fillna(new_value, inplace=True) -> replace cell that is null by new_value
# data.fillna(0, inplace=True) # fill by 0

for col in data.columns:
    data[col] = data[col].fillna(data[col].mean()) # fill by mean (or max, min) value of each column

# data.fillna(data.mean(), inplace=True) # fill by mean of DataFrame (or max, min)
data

# *inplace: sửa trực tiếp trên data gốc

,A,B,C
0,1000,0,3.0
1,400,5,6.0
2,700,6,9.0
3,100,11,12.0
4,1300,0,15.0
5,1600,17,9.0


In [6]:
# sometime, u need to remove 0, ... 
print("check: 0s")
print("=========")
print(data.eq(0).sum())

# thay giá trị 0 bằng giá trị mean của data
data.replace(0, data.mean(), inplace=True)
data

check: 0s
A    0
B    2
C    0
dtype: int64


,A,B,C
0,1000,6.5,3.0
1,400,5.0,6.0
2,700,6.0,9.0
3,100,11.0,12.0
4,1300,6.5,15.0
5,1600,17.0,9.0


## **Step3: Handle duplicate rows**

In [7]:
# .duplicated(keep='Fisrt')
    # - keep='First' (default) -> All duplicates are marked as True except for the first occurrence
    # - keep='Last' -> All duplicates are marked as True except for the last occurrence
    # - keep=False -> All duplicates are marked as True
# data = data[data.duplicated()] # keep first

data.drop_duplicates(keep='first', inplace=True) # -> remove duplicate rows, keep first
#.drop_duplicates(subset=[columns], keep, inplace)

data

,A,B,C
0,1000,6.5,3.0
1,400,5.0,6.0
2,700,6.0,9.0
3,100,11.0,12.0
4,1300,6.5,15.0
5,1600,17.0,9.0


## **Step4: Nomalizing Datatype**

In [8]:
data.dtypes # get datatype of columns

A      int64
B    float64
C    float64
dtype: object

`df[column_name].astype(new_type)` -> convert column A's datatype to new_type -> int, float, str, bool, "category"...   
`df[column_name]  = pd.to_datetime(df[column_name], format="%d %m %Y")` -> convert to datetime  
    

cast multiple column:
`df = df.astype({"A":int, "B": float, C: "datetime64[ns]"})`


## **Step5: Nomalizing String**
`df[column] = df[column].str.method()`  
- `strip()`
- `lower()`, `upper()`
- `replace(',', '')`  
    ..v.v..
    

## **Step6: Nomalizing/Scaling Columns**

In [9]:
# Nomalizing Columns: removing outliers

# Turkey Fences -> IQR
import numpy as np
# outliers is < Q1 - 1.5*IQR or > Q3 + 1.5*IQR
# Q1, Q3 the value in the dataset that holds 25% / 75 of the values below it
# IQR = Q3-Q1

data1 = data.copy()

for column in data1.columns:
    Q1, Q3 = np.percentile(data1[column], [25, 75])
    IQR = Q3-Q1
    _min = Q1-1.5*IQR
    _max = Q3+1.5*IQR
    data1 = data1[(data1[column] >= _min) & (data1[column] <= _max)]

data1

# Ưu điểm: dễ áp dụng, không cần giả định phân phối.
# Nhược điểm: không tốt với dữ liệu không đều, nhiều cực trị.

,A,B,C
0,1000,6.5,3.0
1,400,5.0,6.0
2,700,6.0,9.0
3,100,11.0,12.0
4,1300,6.5,15.0


In [10]:
# based on Z-score

data2 = data.copy()

for column in data2.columns:
    mean = data2.mean()
    std = data2.std()
    z_score = np.abs([(y-mean)/std for y in data2[column]])
    # z_score > 3 or z_score < -3 -> outliers
    data2 = data2[z_score <= 3]

data2
    

,A,B,C
0,1000,6.5,3.0
1,400,5.0,6.0
2,700,6.0,9.0
3,100,11.0,12.0
4,1300,6.5,15.0
5,1600,17.0,9.0


In [11]:
# Scale
# sklearn.preprocessing.MinMaxScaler().fit_transfrom(data.values.astype(float))
# scale all values to the (0, 1) range
from sklearn import preprocessing

# Turkey Fences
x1 = data1.values.astype(float)
# Z_Score
x2 = data2.values.astype(float)

min_max_scaler = preprocessing.MinMaxScaler()
x1_scaled = min_max_scaler.fit_transform(x1)
x2_scaled = min_max_scaler.fit_transform(x2)

print(pd.DataFrame(x1_scaled, columns=data.columns))
print(pd.DataFrame(x2_scaled, columns=data.columns))


      A         B     C
0  0.75  0.250000  0.00
1  0.25  0.000000  0.25
2  0.50  0.166667  0.50
3  0.00  1.000000  0.75
4  1.00  0.250000  1.00
     A         B     C
0  0.6  0.125000  0.00
1  0.2  0.000000  0.25
2  0.4  0.083333  0.50
3  0.0  0.500000  0.75
4  0.8  0.125000  1.00
5  1.0  1.000000  0.50
